In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

In [3]:
import minsearch

In [4]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)

In [ ]:
SELECT * WHERE course='data-engineering-zoomcamp';

In [6]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [12]:
boost = {
    'question': 3.0,
    'section': 0.5    
}

results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [13]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [15]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':q}]
)

In [16]:
response

ChatCompletion(id='chatcmpl-CtbylJZm0z6NfAF4fpFwHWTcYI0pU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether you can still enroll in a course that has already started depends on the specific policies of the institution or platform offering the course. Here are some general steps you can take:\n\n1. **Check the Enrollment Policies:** Visit the course webpage or contact the administration to learn about their policies for late enrollment.\n\n2. **Contact the Instructor:** Reach out to the course instructor to ask if it's possible to join late. They may be willing to accommodate you, especially if you have a valid reason.\n\n3. **Consider the Catch-up Work:** Assess whether you can realistically catch up with the material you’ve missed. Some courses might be more flexible than others.\n\n4. **Look for Enrollment Deadlines:** Some institutions have a cut-off date for adding courses, after which you might need special permission to

In [17]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on the specific policies of the institution or platform offering the course. Here are some general steps you can take:\n\n1. **Check the Enrollment Policies:** Visit the course webpage or contact the administration to learn about their policies for late enrollment.\n\n2. **Contact the Instructor:** Reach out to the course instructor to ask if it's possible to join late. They may be willing to accommodate you, especially if you have a valid reason.\n\n3. **Consider the Catch-up Work:** Assess whether you can realistically catch up with the material you’ve missed. Some courses might be more flexible than others.\n\n4. **Look for Enrollment Deadlines:** Some institutions have a cut-off date for adding courses, after which you might need special permission to enroll.\n\n5. **Ask About Future Sessions:** If you can’t enroll now, inquire about when the course will be offered again, or if there are alternative options 

In [30]:
prompt_template = """
You're a course teaching assisstant. Answer the QUESTION based on the CONTEXT from the FAQ Database.
Use only the fact from the CONTEXT when answering QUESTION.
If the CONTEXT don't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [21]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [23]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [31]:
prompt = prompt_template.format(question=q, context=context).strip()

In [32]:
print(prompt)

You're a course teaching assisstant. Answer the QUESTION based on the CONTEXT from the FAQ Database.
Use only the fact from the CONTEXT when answering QUESTION.
If the CONTEXT don't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final c

In [33]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':prompt}]
)
response.choices[0].message.content

'Yes, you can still enroll after the course has started. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects.'